In [ ]:
# Mendeklarasikan penggnaan TensorFlow
import tensorflow as tf
# Mendeklarsaikan penggunaan NumPy
import numpy as np
# Mendeklarasikan modul os
import os
# Mendeklarasikan modul 'time'
import time

In [ ]:
# Mengunduh file
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


**Load Data**

In [ ]:
# Membaca file dalam bentuk biner
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# Menghitung karakter teks
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
# Memberikan gambaran awal dari teks yang telah dibaca
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
# Mengidentifikasi karakter unik dalam teks
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


**Olah Teks**

Vectorize Teks
Sebelum training, Anda perlu mengonversi string menjadi representasi numerik. tf.keras.layers.StringLookup dapat mengubah setiap karakter menjadi ID numerik. Caranya adalah teks akan dipecah menjadi token terlebih dahulu.

In [ ]:
# Menghasilkan struktur data TensorFlow yang cocok
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
# Mengonversi string menjadi numerik
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

In [ ]:
# Mempresentasikan ID dari karakter 'chars'
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Karena tujuan tutorial ini adalah untuk menghasilkan teks, penting juga untuk membalikkan representasi ini. Untuk ini Anda dapat menggunakan kode tf.keras.layers.StringLookup(..., invert=True).
Catatan: pada kode ini, daripada meneruskan kosakata asli yang dihasilkan dengan diurutkan(set(teks)) gunakan metode get_vocabulary() dari tf.keras.layers.StringLookup sehingga token [UNK] disetel dengan cara yang sama.

In [ ]:
# Mengonversi kembali ID numerik menjadi karakter unik
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

Lapisan ini mengconvert kembali karakter dari vektor ID, dan mengembalikannya sebagai karakter tf.RaggedTensor:

In [ ]:
# Mengembalikan ID numerik yang sesuai dengan mapping yang ditentukan
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Anda dapat menggunakan tf.strings.reduce_join untuk menggabungkan kembali karakter menjadi string.

In [ ]:
# Menggabungkan karakter dari tiap tensor
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
# Mengonversi kumpulan ID numerik dan mengonversi kembali menjadi teks
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

**Prediksi**

Diberikan sebuah karakter, atau serangkaian karakter, karakter apa yang paling mungkin berikutnya? Ini adalah tugas yang harus Anda latih agar model dapat melakukannya. Masukan ke model akan berupa urutan karakter, dan Anda melatih model untuk memprediksi keluaran berupa karakter berikut pada setiap langkah waktu. Karena RNN mempertahankan keadaan internal yang bergantung pada elemen yang terlihat sebelumnya, mengingat semua karakter dihitung hingga saat ini, karakter apa selanjutnya?

In [ ]:
# Mengonversi teks menjadi ID numerik
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
# Melatih model untuk pemrosesan data lanjutan
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
# Menggabungkan karakter unik pada setiap elemen dataset
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [ ]:
# Menyimpan panjang urutan
seq_length = 100

In [ ]:
# Menunjukkan jika panjang urutan tidak dapat dibagi habis, maka elemen yang tersisa akan dihapus
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# Melakukan iterasi
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
# Mengonversi hasil setiap ID numerik menjadi array NumPy
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
# Memprediksi elemen
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
# Memisahkan list menjadi 2 bagian
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
# Memisahkan setiap urutan ID numerik menjadi pasangan input
dataset = sequences.map(split_input_target)

In [ ]:
# Mencetak pasangan input dan target untuk melihat urutan ID yang konkret
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


**Membuat Batch Training**

Anda menggunakan tf.data untuk membagi teks menjadi sequence yang dapat diatur. Namun sebelum memasukkan data ini ke dalam model, Anda perlu mengacak data dan mengemasnya ke dalam batch.

In [ ]:
# Menentukan ukuran batch yang akan digunakan
BATCH_SIZE = 64

# Menentukan ukuran buffer yang digunakan saat melakukan pengacakan dataset
BUFFER_SIZE = 10000

#
dataset = (
    dataset
    # Melakukan pengacakan elemen dalam dataset menggunakan buffer seukuran BUFFER_SIZE
    .shuffle(BUFFER_SIZE)
    # Membagi dataset menjadi batch dengan ukuran BATCH_SIZE
    .batch(BATCH_SIZE, drop_remainder=True)
    # Menerapkan prefetching pada dataset
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

**Buat Model**

In [ ]:
# Menghitung panjang atau jumlah elemen dalam vocabulary
vocab_size = len(ids_from_chars.get_vocabulary())

# Menentukan dimensi embedding
embedding_dim = 256

# Menentukan jumlah unit untuk memproses urutan data
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    # Mengonversi ID numerik menjadi representasi vektor kontinu
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # Memproses urutan data dengan menentukan jumlah unit GRU
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    # Menentukan jumlah unit output
    self.dense = tf.keras.layers.Dense(vocab_size)

  # Mendefinisikan bagaimana model melakukan forward pass
  def call(self, inputs, states=None, return_state=False, training=False):
    # Menginisialisasi variabel x
    x = inputs
    # Memproses input melalui layer embedding
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    # Memproses input melalui layer GRU
    x, states = self.gru(x, initial_state=states, training=training)
    # Memproses hasil dari GRU melalui layer Dense untuk mendapatkan output model
    x = self.dense(x, training=training)

    # mengembalikan output (x)
    if return_state:
      return x, states
    # Jika tidak, model hanya akan mengembalikan output (x).
    else:
      return x

In [ ]:
model = MyModel(
    # Menentukan ukuran vocabulary
    vocab_size=vocab_size,
    # Menentukan dimensi embedding
    embedding_dim=embedding_dim,
    # Menentukan jumlah unit dalam layer GRU
    rnn_units=rnn_units)

**Uji Model**

In [ ]:
# Melakukan iterasi melalui satu batch pertama dari dataset
for input_example_batch, target_example_batch in dataset.take(1):
    # Membuat prediksi pada input batch
    example_batch_predictions = model(input_example_batch)
    # Mencetak bentuk dari hasil prediksi
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
# Mencetak summary
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Menghasilkan sampel indeks
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
# Mengekstrak dan menampilkan nilai indeks
sampled_indices

array([12, 53, 13,  9, 18, 34, 34,  7, 46, 18, 21, 63, 20, 29, 30, 36, 59,
       40, 32, 50, 57, 62, 33, 26,  2,  4, 47, 65, 50, 63, 25, 47, 23, 22,
        5, 46, 34, 12, 25, 10, 54, 16, 57, 52, 60, 28,  5, 14, 62, 26, 50,
       55, 47, 54, 51, 30, 18, 36, 59, 51, 29, 49, 27, 44,  2, 64,  3, 24,
       10, 39, 14, 53, 51,  7, 61, 50, 44,  2, 25, 14, 56, 11, 10, 43, 16,
       53, 26, 17, 39, 57, 39, 46, 14,  1, 25, 43, 51, 42, 42, 63])

In [ ]:
# Mencetak teks dari input ke konsol dengan label "Input"
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
# Mencetak teks dari sampel indeks ke konsol dengan label "Next Char Predictions"
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'Third Servingman:\nO slaves, I can tell you news,-- news, you rascals!\n\nFirst Servingman:\nWhat, what,'

Next Char Predictions:
 b';n?.EUU,gEHxGPQWtaSkrwTM $hzkxLhJI&gU;L3oCrmuO&AwMkpholQEWtlPjNe y!K3ZAnl,vke LAq:3dCnMDZrZgA\nLdlccx'


**Train Model**

Pada titik ini permasalahan dapat dianggap sebagai permasalahan klasifikasi standar. Permasalahan dapat disimpulkan dengan : Berdasarkan status RNN sebelumnya, dan masukan langkah kali ini, prediksi kelas karakter berikutnya.

In [ ]:
# Mendefinisikan objek fungsi kerugian untuk menghitung kerugian antara prediksi model dan label yang sebenarnya
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Menghasilkan nilai rata-rata kerugian untuk satu batch
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
# Mencetak bentuk dari prediksi model
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
# Mencetak nilai rata-rata kerugian untuk satu batch
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1889763, shape=(), dtype=float32)


In [ ]:
# Menghitung nilai eksponensial dari setiap elemen dalam tensor
tf.exp(example_batch_mean_loss).numpy()

65.95524

In [ ]:
# Menentukan optimizer yang akan digunakan selama pelatihan
model.compile(optimizer='adam', loss=loss)

**Konfigurasi Checkpoints**

Gunakan tf.keras.callbacks.ModelCheckpoint untuk memastikan bahwa checkpoint disimpan selama pelatihan:

In [ ]:
# Menentukan direktori tempat checkpoint akan disimpan
checkpoint_dir = './training_checkpoints'
# Menentukan awalan nama file untuk checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    # Menentukan jalur dan awalan nama file untuk checkpoint
    filepath=checkpoint_prefix,
    # Menentukan bahwa hanya bobot model yang akan disimpan sebagai checkpoint
    save_weights_only=True)

**Lakukan Proses Training**

Agar waktu pelatihan tidak terlalu lama, gunakan 10 epoch untuk melatih model. Di Colab, setel runtime ke GPU untuk pelatihan yang lebih cepat.

In [ ]:
# Menentukan jumlah epoch
EPOCHS = 10

In [ ]:
# Melatih dengan dataset yang diberikan
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 1045s 6s/step - loss: 2.6982
Epoch 2/10
172/172 [==============================] - 1043s 6s/step - loss: 1.9715
Epoch 3/10
172/172 [==============================] - 1031s 6s/step - loss: 1.6931
Epoch 4/10
172/172 [==============================] - 1021s 6s/step - loss: 1.5357
Epoch 5/10
172/172 [==============================] - 1022s 6s/step - loss: 1.4389
Epoch 6/10
172/172 [==============================] - 1026s 6s/step - loss: 1.3710
Epoch 7/10
172/172 [==============================] - 1010s 6s/step - loss: 1.3200
Epoch 8/10
172/172 [==============================] - 1030s 6s/step - loss: 1.2742
Epoch 9/10
172/172 [==============================] - 1007s 6s/step - loss: 1.2327
Epoch 10/10
172/172 [==============================] - 1009s 6s/step - loss: 1.1926


**Generate Teks**

In [ ]:
class OneStep(tf.keras.Model):
  # Mengontrol sejauh mana variasi output yang dihasilkan oleh model
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    # Menyimpan model sebagai atribut kelas
    self.model = model
    # Menyimpan fungsi konversi karakter
    self.chars_from_ids = chars_from_ids
    # Menyimpan fungsi konversi token
    self.ids_from_chars = ids_from_chars

    # Membuat tensor yang berisi token ID dari karakter [UNK]
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    # Membangun tensor yang banyak nilainya nol
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  #  Mendeklarasikan metode generate_one_step sebagai fungsi TensorFlow yang dapat dijalankan secara grafis
  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Mengonversi input ke token IDs
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    # Mengonversi karakter input ke token IDs dan mengonversi hasilnya ke dalam bentuk tensor
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Menjalankan model untuk mendapatkan logits dan state model
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Mengambil prediksi terakhir
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Menerapkan mask pada logits untuk mencegah karakter [UNK]
    predicted_logits = predicted_logits + self.prediction_mask

    # Mengambil sampel dari distribusi kategorikal untuk mendapatkan token IDs
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    #Menghilangkan dimensi ke-2 dari tensor hasil sampel
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Mengonversi token IDs hasil sampel ke karakter
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Mengembalikan karakter
    return predicted_chars, states

In [ ]:
# Mengonversi karakter ke token IDs untuk mengonversi input karakter menjadi token IDs
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
# Merekam waktu awal sebelum proses generasi teks dimulai
start = time.time()
states = None
# Menentukan karakter awal yang akan menjadi input pertama untuk model generatif
next_char = tf.constant(['ROMEO:'])
result = [next_char]

# Melakukan Loop sebanyak 1000 langkah untuk menghasilkan teks berisi 1000 karakter
for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  # Menambahkan karakter yang dihasilkan ke dalam list result
  result.append(next_char)

# Menggabungkan semua karakter yang dihasilkan dalam 1 string
result = tf.strings.join(result)
# Merekam waktu akhir setelah proses generasi teks selesai
end = time.time()
# Mencetak teks
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
And I will own to-morrow.

Clown:
Ah, ay, you are heard.

GREGORY:
They say the corn, that's created. Conceives my blood
In steps hur discockers I chaple ye; sir,
what he you new crept of holds. By him affect.

Second Citizen:
The shadow paunt of women are you quarrel, for youe
that thou sursed state and wanders him;
And with Carthag's breathed in this sland
To see the flouring penarts to be all
gred forth confused with the execution
Should bring it could see the affice of my request.
Now in the warlike seems like enficious he
make me no were to them drop to me.

MARCIUS:
Thou love's meet accept.

QUEEN ELIZABETH:
And will not he to cloof, he fall not join;
For neither death in warwhed to your own course,
Blow thrush my head is dukedom and in that dif.
A kingly dear spitchet and cracked in;
Which would be resosted with the state: where thou satest
to still unjasting by the comfort and ceres
And feat protectors by me, and more thou, man in steel
One throwed out of necessable.
Abo

In [ ]:
# Merekam waktu awal sebelum proses generasi teks dimulai
start = time.time()
states = None
# Menentukan beberapa string awal sebagai input
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

# Melakukan Loop sebanyak 1000 langkah untuk menghasilkan teks berisi 1000 karakter
for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

# Menggabungkan semua string yang dihasilkan menjadi satu string
result = tf.strings.join(result)
# Merekam waktu akhir setelah proses generasi teks selesai
end = time.time()
# Mencetak teks
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThey must partain Coponior's comfort-back burns in\nRecents of holds me. Shall, I reason to thee:\nThy noble majesty doth my reckence?\nNow, sir, little good-horse grass and but what tyrant\nTo hear himself leading us I see\n, ender im not to be her manner.\n\nNORFOLK:\nAlas, thou face the issue?\n\nSTCAMILLO:\nSir, be right;\nLet him well so, if defore I of your redempts\nWhen you most unturnible sought deceived.\nBut what often and there to armed and good?\n\nRICHARD:\nO, you may do what care?\n\nPOMPEY:\nWell said, you, farewell; let'? all heart\nMy brother, didst peace and wept the drop.\n\nGLOUCESTER:\nThey say, a very sixternoon to my renowned than in,\nTo--fortune, to execute of friar, toop for this\nTo put it offrighty-ornated, answerlious frowns;\nAnd be you seement, young and all the lew\nThat, my diffect of the noble direction\nIf ever mine own fatterngary where thou\nDauet'st, scalls with another waitings. But what, a boundy\nTo emils, my house: marry,

**Ekspor Model Generator**

Model satu langkah ini dapat dengan mudah disimpan dan digunakan kembali, memungkinkan Anda menggunakannya di mana pun tf.saved_model diterima.

In [ ]:
# Menyimpan model ke direktori dengan nama 'one_step'
tf.saved_model.save(one_step_model, 'one_step')
# Memuat kembali model dari direktori yang sama, 'one_step'
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
#  Inisialisasi state model
states = None
# Menentukan karakter awal
next_char = tf.constant(['ROMEO:'])
# Melakukan nisialisasi list result
result = [next_char]

# Loop sebanyak 100 langkah untuk menghasilkan teks berisi 100 karakter
for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)
# Menggabungkan semua karakter yang dihasilkan menjadi satu string dan mencetaknya
print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
What, you have offly soul!

First Senator:
Hath can be you deeds, ready by consent
Ah, goien: sit t


# Tugas

*   Jalankan Model dan hitung loss dengan tf.GradientTape.
*   Hitung update dan terapkan pada model dengan optimizer



In [ ]:
class CustomTraining(MyModel):
     @tf.function
     def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
            grads = tape.gradient(loss, model.trainable_variables)
            self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

            return {'loss': loss}


In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))


In [ ]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 949s 5s/step - loss: 2.7218


In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)


    if (epoch + 1) % 5 == 0:
      model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))


Epoch 1 Batch 0 Loss 2.1924
Epoch 1 Batch 50 Loss 2.0588
Epoch 1 Batch 100 Loss 1.9927
Epoch 1 Batch 150 Loss 1.8179

Epoch 1 Loss: 1.9860
Time taken for 1 epoch 921.91 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7544
Epoch 2 Batch 50 Loss 1.7466
Epoch 2 Batch 100 Loss 1.6997
Epoch 2 Batch 150 Loss 1.6337

Epoch 2 Loss: 1.7116
Time taken for 1 epoch 921.92 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5910
Epoch 3 Batch 50 Loss 1.6010
Epoch 3 Batch 100 Loss 1.5403
Epoch 3 Batch 150 Loss 1.5356

Epoch 3 Loss: 1.5518
Time taken for 1 epoch 921.91 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4751
Epoch 4 Batch 50 Loss 1.4134
Epoch 4 Batch 100 Loss 1.4284
Epoch 4 Batch 150 Loss 1.4634

Epoch 4 Loss: 1.4519
Time taken for 1 epoch 921.92 sec
_________________________________________________________________

*   Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?


Tugas diatas menggunakan tf.GradientTape yang mana pada gradien dari nilai kesalahan tersebut dapat dilatih dalam model dilacak, kemudian diterapkan untuk memperbarui variabel tersebut. Prediksi model ini didasari oleh hasil prediksi sebelumnya pada langkah sebelumnya. Sehingga, apabila terdapat kesalahan dalam prediksi, akan berdampak pada langkah berikutnya. Jadi, setiap kesalahan pada setiap langkah akan memengaruhi hasil prediksi model.